In [230]:
import pandas as pd
from collections import Counter 
from helper import load_data,pipeline, unique_drugs, generate_drugs_df
import re


%pylab inline 
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [231]:
trials = load_data()
c = unique_drugs(trials)
df, exceptions = generate_drugs_df(c, trials)
df

got 127 results and had 0 exceptions
probs are[83.46, 58.49, 58.06]


,drug,num_of_trials,phase_1_success,phase_2_success,phase_3_success,phase_1_2_success,phase_2_3_success,phase_1_start,phase_1_first_end,phase_1_last_end,phase_2_start,phase_2_first_end,phase_2_last_end
14,Trastuzumab,441,True,True,True,True,True,2002-04-01,2002-05-01,2019-03-01,1998-04-01,1999-09-01,2019-04-01
3,Paclitaxel,269,True,True,True,True,True,2005-12-01,2006-01-25,2019-01-24,1998-04-01,1999-11-01,2019-01-25
56,Lapatinib,181,True,True,True,True,True,2003-04-28,2003-11-01,2016-01-06,2002-10-01,2003-12-09,2018-05-01
22,Capecitabine,175,True,True,True,True,True,2000-08-01,2001-11-01,2019-01-31,1998-07-01,2001-09-01,2019-03-15
2,Bevacizumab,167,True,True,True,True,False,2005-09-01,2005-10-01,2015-11-23,2001-11-01,2001-12-01,2018-07-01
28,Eribulin,149,True,True,True,True,False,2011-10-01,2011-11-01,2018-05-30,2004-09-01,2006-09-20,2019-01-01
9,Palbociclib,140,True,True,True,True,True,2011-03-01,2012-02-01,2019-03-01,2009-10-01,2013-02-22,2019-03-01
60,Docetaxel,136,True,True,True,True,True,1996-12-01,1997-01-01,2011-01-01,1997-01-01,1997-06-01,2019-03-01
20,Letrozole,132,True,True,True,True,True,2010-03-01,2010-09-01,2018-12-11,2000-01-01,2000-07-26,2019-04-01
30,Everolimus,125,True,True,True,True,False,2005-11-01,2006-08-16,2017-07-17,2005-01-19,2005-08-03,2018-11-28


## Error handling to talk w/ Adam about

In [237]:
df = trials[trials['Primary Drugs'].str.contains(
        'Nivolumab', case=False)]
df['trial_start'] = df.apply(start_to_datetime, axis=1)
df['trial_end'] = df.apply(end_to_datetime, axis=1)
if  numpy.datetime64('NaT') in df[df['Phase of Trial'] == 'Phase I']['trial_start'].values:
    print ('hi')

#df[df['Phase of Trial'] == 'Phase I']['trial_start'].values[0]
# df[df['Phase of Trial'] == 'Phase I']['Trial Initiation date'].values


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


Phase II      19
Phase I/II    10
Phase I       10
Phase III      1
Phase IV       1
Name: Phase of Trial, dtype: int64

In [244]:
df[df['Phase of Trial']=='Phase IV']['Trial Initiation date']

507    15 Jan 2018 (actual)
Name: Trial Initiation date, dtype: object

In [243]:
prob_of_success(df)

[True, True, True, True, False]